In [1]:
import argparse
import os
import numpy as np
from gensim import models
from keras.callbacks import ModelCheckpoint, LambdaCallback, TensorBoard
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding, CuDNNLSTM, Bidirectional
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
import pickle

/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
caminho_modelo_w2v = "modelos/w2v_sertanejo_28kmusicas_tweet_tknzr_window_15_mincount_10.model"
if not os.path.isfile(caminho_modelo_w2v):
    raise FileExistsError("Modelo w2v nao encontrado.")

In [4]:
w2v_model = models.Word2Vec.load(caminho_modelo_w2v)
vocab = list(w2v_model.wv.vocab)

In [9]:
len(vocab)

12082

In [10]:
def word2idx(word):
    return w2v_model.wv.vocab[word].index


def idx2word(idx):
    return w2v_model.wv.index2word[idx]

In [17]:
nome_modelo = "lstm-w2v-wordlevel-15len-512-256-sertanejo.model"
caminho_modelo_lstm = "modelos/{}".format(nome_modelo)

In [18]:
if not os.path.isfile(caminho_modelo_lstm):
    raise FileExistsError("Modelo RNN não encontrado")

print("Carregando modelo lstm previo...")
model = load_model(caminho_modelo_lstm)

Carregando modelo lstm previo...


/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/sklearn/utils/fixes.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if 'order' in inspect.getargspec(np.copy)[0]:


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[Node: cu_dnnlstm_1/CudnnRNN = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](cu_dnnlstm_1/transpose, cu_dnnlstm_1/ExpandDims_1, cu_dnnlstm_1/ExpandDims_2, cu_dnnlstm_1/concat_1)]]

Caused by op 'cu_dnnlstm_1/CudnnRNN', defined at:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/daniel/dev/venvs/p36/l

In [19]:
def gerar_texto(seed, temperatura):

    tamanho_sentencas = 15
    seed_tokens = [word2idx(i) for i in seed.split()]
    print("Seed:")
    print("\"", seed, "\"\n")

    aux = 1
    for i in range(500):

        xt = np.zeros((1, tamanho_sentencas), dtype=np.int32)

        for t, token in enumerate(seed_tokens):
            xt[0, t] = token

        prediction = model.predict(xt, verbose=0)

        if temperatura > 0 and aux % temperatura == 0:
            ordered = (-prediction[0]).argsort()[:3]
            indice = np.random.choice(ordered)
        else:
            indice = (np.argmax(prediction))

        result = idx2word(indice)
        print("\n" if result == "nlinha" else result, end=" ")
        seed_tokens.append(result)
        seed_tokens = seed_tokens[1:len(seed_tokens)]
        aux = aux + 1

    print("\n\nFIM\n\n", )

In [20]:
frase_seed = "quem foi que disse que nao existe sertanejo na capital federal nlinha ta passando"
temperatura = 5
gerar_texto(frase_seed,temperatura)

Seed:
" quem foi que disse que nao existe sertanejo na capital federal nlinha ta passando "



NameError: name 'model' is not defined